In [34]:
import pandas as pd
import numpy as np
import os as os
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as MSE

### Load data

In [3]:
dfs_train = {}
dfs_test = {}

In [4]:
for root, directory, files in os.walk("data/train_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_train[file[:-4]] = pd.read_csv(f"{root}/{file}")
for root, directory, files in os.walk("data/test_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_test[file[:-4]] = pd.read_csv(f"{root}/{file}")
            dfs_test[file[:-4]].drop(columns=["Target", "Target_clas", "Target_regr"], inplace=True)

In [5]:
def impute_nan_values(dfs):
    for df in dfs.values():
        for col in df.columns:
            if df[col].dtype == np.float64 or df[col].dtype == np.int64:
                df[col] = df.groupby("season")[col].transform(lambda x: x.fillna(x.mean()))
        df.dropna(inplace=True)
impute_nan_values(dfs_train)
impute_nan_values(dfs_test)

In [6]:
# validation set

dfs_valid_reg_X = {}
dfs_valid_reg_y = {}
dfs_train_reg_X = {}
dfs_train_reg_y = {}
dfs_valid_clas_X = {}
dfs_valid_clas_y = {}
dfs_train_clas_X = {}
dfs_train_clas_y = {}


cols_to_drop = ['FTHG', 'FTAG', 'MatchTeams', 'SameHomeTeam', 'Target', 'Target_regr', 'Target_clas']

for country in dfs_train:
    dfs_valid_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_regr"]
    dfs_valid_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop)

    dfs_train_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_regr"]
    dfs_train_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop)

    dfs_train_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_clas"]
    dfs_train_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop)

    dfs_valid_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_clas"]
    dfs_valid_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop)

### Regression task


baseline model


In [7]:
bl_forests = {}

In [8]:
for country in dfs_train_reg_X:
    bl_forests[country] = RandomForestRegressor(n_estimators=100, criterion="squared_error", max_depth=10)

In [13]:
pd.set_option('display.max_columns', 500)

In [42]:
for country in bl_forests:
    try:
        dfs_valid_reg_X[country].drop(columns=['Avg_bookie_prediction'], axis=1, inplace=True)
        dfs_train_reg_X[country].drop(columns=['Avg_bookie_prediction'], axis=1, inplace=True)
    except:
        print("hmm")

hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm


In [28]:
dfs_train_reg_X['england'].head()

,Unnamed: 0,season,Avg_away_odds,Avg_home_odds,Avg_draw_odds,Var_away_odds,Var_home_odds,Var_draw_odds,LastMatchAwayGoals,LastMatchHomeGoals,LastMatchAwayWin,LastMatchHomeWin,LastMatchDraw,HomeTeam_AFC Wimbledon,HomeTeam_Accrington,HomeTeam_Aldershot,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Barnet,HomeTeam_Barnsley,HomeTeam_Barrow,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Boston,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Bristol City,HomeTeam_Bristol Rvs,HomeTeam_Burnley,HomeTeam_Burton,HomeTeam_Bury,HomeTeam_Cambridge,HomeTeam_Cardiff,HomeTeam_Carlisle,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Cheltenham,HomeTeam_Chester,HomeTeam_Chesterfield,HomeTeam_Colchester,HomeTeam_Coventry,HomeTeam_Crawley Town,HomeTeam_Crewe,HomeTeam_Crystal Palace,HomeTeam_Dag and Red,HomeTeam_Darlington,HomeTeam_Derby,HomeTeam_Doncaster,HomeTeam_Everton,HomeTeam_Exeter,HomeTeam_Fleetwood Town,HomeTeam_Forest Green,HomeTeam_Fulham,HomeTeam_Gillingham,HomeTeam_Grimsby,HomeTeam_Halifax,HomeTeam_Harrogate,HomeTeam_Hartlepool,HomeTeam_Hereford,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Kidderminster,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Leyton Orient,HomeTeam_Lincoln,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Macclesfield,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Mansfield,HomeTeam_Middlesbrough,HomeTeam_Millwall,HomeTeam_Milton Keynes Dons,HomeTeam_Morecambe,HomeTeam_Newcastle,HomeTeam_Newport County,HomeTeam_Northampton,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Notts County,HomeTeam_Oldham,HomeTeam_Oxford,HomeTeam_Peterboro,HomeTeam_Plymouth,HomeTeam_Port Vale,HomeTeam_Portsmouth,HomeTeam_Preston,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Rochdale,HomeTeam_Rotherham,HomeTeam_Rushden & D,HomeTeam_Salford,HomeTeam_Scunthorpe,HomeTeam_Sheffield United,HomeTeam_Sheffield Weds,HomeTeam_Shrewsbury,HomeTeam_Southampton,HomeTeam_Southend,HomeTeam_Stevenage,HomeTeam_Stockport,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Sutton,HomeTeam_Swansea,HomeTeam_Swindon,HomeTeam_Torquay,HomeTeam_Tottenham,HomeTeam_Tranmere,HomeTeam_Walsall,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wimbledon,HomeTeam_Wolves,HomeTeam_Wrexham,HomeTeam_Wycombe,HomeTeam_Yeovil,HomeTeam_York,AwayTeam_AFC Wimbledon,AwayTeam_Accrington,AwayTeam_Aldershot,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Barnet,AwayTeam_Barnsley,AwayTeam_Barrow,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Boston,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Bristol City,AwayTeam_Bristol Rvs,AwayTeam_Burnley,AwayTeam_Burton,AwayTeam_Bury,AwayTeam_Cambridge,AwayTeam_Cardiff,AwayTeam_Carlisle,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Cheltenham,AwayTeam_Chester,AwayTeam_Chesterfield,AwayTeam_Colchester,AwayTeam_Coventry,AwayTeam_Crawley Town,AwayTeam_Crewe,AwayTeam_Crystal Palace,AwayTeam_Dag and Red,AwayTeam_Darlington,AwayTeam_Derby,AwayTeam_Doncaster,AwayTeam_Everton,AwayTeam_Exeter,AwayTeam_Fleetwood Town,AwayTeam_Forest Green,AwayTeam_Fulham,AwayTeam_Gillingham,AwayTeam_Grimsby,AwayTeam_Halifax,AwayTeam_Harrogate,AwayTeam_Hartlepool,AwayTeam_Hereford,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Kidderminster,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Leyton Orient,AwayTeam_Lincoln,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Macclesfield,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Mansfield,AwayTeam_Middlesbrough,AwayTeam_Millwall,AwayTeam_Milton Keynes Dons,AwayTeam_Morecambe,AwayTeam_Newcastle,AwayTeam_Newport County,AwayTeam_Northampton,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Notts County,AwayTeam_Oldham,AwayTeam_Oxford,AwayTeam_Peterboro,AwayTeam_Plymouth,AwayTeam_Port Vale,AwayTeam_Portsmouth,AwayTeam_Preston,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Rochdale,AwayTeam_Rotherham,AwayTeam_Rushden & D,AwayTeam_Salford,AwayTeam_Scunthorpe,AwayTeam_Sheffield

In [32]:
for country in bl_forests:
    print("fitting ", country)
    bl_forests[country].fit(X=dfs_train_reg_X[country], y=dfs_train_reg_y[country])

belgium
england
france
germany
greece
italy
netherlands
portugal
scotland
spain
turkey


In [44]:
for country in bl_forests:
    # print("predicting ", country)
    prediction = bl_forests[country].predict(dfs_valid_reg_X[country])
    print("mse of ", country, ": ", MSE(prediction, dfs_valid_reg_y[country]))
    

mse of  belgium :  3.847979612786379
mse of  england :  2.5258497623522596
mse of  france :  2.6276809887076014
mse of  germany :  2.890467457862476
mse of  greece :  2.6686605750107346
mse of  italy :  2.5276304656145836
mse of  netherlands :  2.6629481826279977
mse of  portugal :  2.2096146182599226
mse of  scotland :  2.505359775518794
mse of  spain :  2.5666686238669496
mse of  turkey :  3.0266036377871366


random search


grid search


train on best params


*repeat with boosting?*